In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
from implementations import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [33]:
#Check data
tX_list, ids_list = separate_dataset(tX, ids)
index=[]
for indexX, column in enumerate(tX_list[0].T):
    x_= np.mean(column)
    for indexY, column2 in enumerate(tX_list[0].T):
        y_= np.mean(column2)
        num=np.dot((column - x_),(column2 - y_))
        den= np.sqrt(np.dot(column - x_,column - x_)*np.dot(column2 - y_,column2 - y_))
        corr= num/den
        if corr> 0.8 and indexX!= indexY:
            print(indexX, indexY)
        



ValueError: shapes (99913,) and (250000,) not aligned: 99913 (dim 0) != 250000 (dim 0)

In [107]:
# Ridge regression with K-fold
k_fold= 4
degrees = range(1, 3)
lambdas = np.logspace(-4, 0, 30)
seed = 1
#ratio_train = 0.7

rmse_te = 0
BestDeg=0
BestLambda=0
#x_tr, x_te, y_tr, y_te= split_data(tX, y, ratio_train, seed)
k_indices =build_k_indices(y, k_fold, seed)
rmse_te_tmp = []
BestLambdaForDeg=[]
for index_degree, degree in enumerate(degrees):
    rmse_te_tmp2 = []
    for index_lambda, lambda_ in enumerate(lambdas):
        loss_te_tmp=0
        for k in range(k_fold):
            _, loss_te, w=cross_validation(y, tX, k_indices, k, lambda_, degree)
            loss_te_tmp= loss_te_tmp+loss_te
        loss_te_tmp=loss_te_tmp/k_fold
        rmse_te_tmp2.append(np.sqrt(2*loss_te_tmp))
    rmse_te_tmp.append(min(rmse_te_tmp2))
    BestLambdaForDeg.append(lambdas[np.argmin(rmse_te_tmp2)])
BestDeg=degrees[np.argmin(rmse_te_tmp)]
BestLambda= BestLambdaForDeg[np.argmin(rmse_te_tmp)]
rmse_te=min(rmse_te_tmp)


loss=rmse_te
print(BestDeg)
print(BestLambda)


LinAlgError: Singular matrix

In [ ]:
# Least squares regression using normal equation
weight, loss = least_squares(y, tX)
loss

0.33968680947709307

In [104]:
# Ridge regression using normal equations
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
lambda_ = BestLambda
tx=build_poly(tX,BestDeg)

weights, loss = ridge_regression(y, tx, lambda_)
loss

0.33944686042508954

## Generate predictions and save ouput in csv format for submission:

In [105]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_test_poly=build_poly(tX_test,BestDeg)

In [106]:
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test_poly)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)